In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
    
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process.kernels import RBF
from sklearn.ensemble import AdaBoostClassifier
from sklearn.gaussian_process import GaussianProcessClassifier

%matplotlib inline

In [2]:
dict_classifiers = {
    "Logistic Regression": LogisticRegression(),
    "Nearest Neighbors": KNeighborsClassifier(3),
    "Linear SVM": SVC(kernel="linear", C=0.025),
    "Gradient Boosting Classifier": GradientBoostingClassifier(n_estimators=1000),
    "Decision Tree": tree.DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(n_estimators=1000),
    "Neural Net": MLPClassifier(alpha = 1),
    "Naive Bayes": GaussianNB(),
    "AdaBoost": AdaBoostClassifier(),
    "Gaussian Process": GaussianProcessClassifier()
}

In [3]:
def LoadData(negFilePath, posFilePath):
    negFileContent = pd.read_csv(negFilePath)
    posFileContent = pd.read_csv(posFilePath)
    df = negFileContent.append(posFileContent)
    df = shuffle(df)
    del negFileContent
    del posFileContent
    return df

In [4]:
def SplitData(df):
    # get features and labels
    featuresColumns = df.columns[0:len(df.columns) - 1]
    lablesColumn = df.columns[len(df.columns) - 1]
    trainingFeaturesX = df.loc[:, featuresColumns]
    trainingLabely = df.loc[:, lablesColumn]
    # split the data into training and test
    traningFeatures, testFeatures, trainingLabels, actualResults = train_test_split(trainingFeaturesX, trainingLabely,test_size=0.33, random_state=42)
    return traningFeatures, testFeatures, trainingLabels, actualResults

In [5]:
def CAR_Filter(df):
    dfCAR = pd.DataFrame().reindex_like(df)
    for j in range (df.shape[1]-1):
        avg = df.iloc[:,j].mean()
        for i in range (df.shape[0]):
            dfCAR.iloc[i,j] = df.iloc[i,j] - avg
    dfCAR.iloc[:,df.shape[1]-1] = pd.Series(df.iloc[:,df.shape[1]-1])
    return dfCAR

In [6]:
def batch_classify(X_train, Y_train, X_test, Y_test, no_classifiers = 1):

    for classifier_name, classifier in list(dict_classifiers.items())[:no_classifiers]:
        classifier.fit(X_train, Y_train)
        Y_pred = classifier.predict(X_test)
        
        cm = confusion_matrix(Y_test, Y_pred)    
        print (classifier_name)
        print (cm)
        fpr, tpr, thresholds = metrics.roc_curve(Y_test, Y_pred)
        auc = metrics.auc(fpr, tpr)
        print (auc)
        
        #fig = plt.figure()
        #ax = fig.add_subplot(111)
        #cax = ax.matshow(cm)
        #fig.colorbar(cax)
        #labels= ['no brake signal', 'brake signal']
        #ax.set_xticklabels([''] + labels)
        #ax.set_yticklabels([''] + labels)
        #plt.xlabel('Predicted')
        #plt.ylabel('True')
        #plt.show()

In [7]:
fnames = ["Fatema", "Yousef", "Hesham", "Tair2", "Mohamed EL-Tair", "Mohamed Wagih", "Wagih2", "Hesham2"]

In [8]:
for i in range (8):
    df = LoadData("csv/" + fnames[i] + "neg" + ".csv", "csv/" + fnames[i] + "pos" + ".csv")
    print ("=================================================================")
    print ("=================================================================")
    print ("Results for ", fnames[i])
    print (df.shape)
    print ("=================================================================")
    print ("=================================================================")
    
    
    traningFeatures, testFeatures, trainingLabels, actualResults = SplitData(df)
    print ("=================================================================")
    print ("Training 10 different classifiers on raw data")
    print ("=================================================================")
    dictModels = batch_classify(traningFeatures, trainingLabels, testFeatures, actualResults, no_classifiers=10)
    
    
    df_CAR = CAR_Filter(df)
    print (df.shape)
    traningFeatures_CAR, testFeatures_CAR, trainingLabels_CAR, actualResults_CAR = SplitData(df_CAR)
    print ("=================================================================")
    print ("Training 10 different classifiers on data after applying CAR filter")
    print ("=================================================================")
    dictModels = batch_classify(traningFeatures_CAR, trainingLabels_CAR, testFeatures_CAR, actualResults_CAR, no_classifiers=10)
    
    print ("=================================================================")
    print ("Training 10 different classifiers, and different PCA components without CAR filter")
    print ("=================================================================")
    scaler = StandardScaler()
    # Fit on training set only.
    scaler.fit(traningFeatures)
    # Apply transform to both the training set and the test set.
    train_img = scaler.transform(traningFeatures)
    test_img = scaler.transform(testFeatures)
    for i in range (15, traningFeatures.shape[0], 15):
        pca = PCA(n_components=i)
        pca.fit(train_img)
        PCA_training_features = pca.transform(train_img)
        PCA_testing_features = pca.transform(test_img)
        print ("=================================================================")
        print ("Results for ", i, " PCA components")
        print ("=================================================================")
        batch_classify(PCA_training_features, trainingLabels, PCA_testing_features, actualResults, no_classifiers=10)
        
    print ("=================================================================")
    print ("Training 10 different classifiers, and different PCA components after CAR filter")
    print ("=================================================================")
    scaler = StandardScaler()
    # Fit on training set only.
    scaler.fit(traningFeatures)
    # Apply transform to both the training set and the test set.
    train_img = scaler.transform(traningFeatures_CAR)
    test_img = scaler.transform(testFeatures_CAR)
    for i in range (15, traningFeatures_CAR.shape[0], 15):
        pca = PCA(n_components=i)
        pca.fit(train_img)
        PCA_training_features = pca.transform(train_img)
        PCA_testing_features = pca.transform(test_img)
        print ("=================================================================")
        print ("Results for ", i, " PCA components")
        print ("=================================================================")
        batch_classify(PCA_training_features, trainingLabels_CAR, PCA_testing_features, actualResults_CAR, no_classifiers=10)

Results for  Fatema
(599, 2689)
Training 10 different classifiers on raw data


C:\Users\Nour Al-Hoda\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Nour Al-Hoda\Anaconda3\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Logistic Regression
[[187   2]
 [  3   6]]
0.828042328042328
Nearest Neighbors
[[189   0]
 [  9   0]]
0.5
Linear SVM
[[187   2]
 [  3   6]]
0.828042328042328
Gradient Boosting Classifier
[[186   3]
 [  6   3]]
0.6587301587301587
Decision Tree
[[182   7]
 [  6   3]]
0.6481481481481481
Random Forest
[[187   2]
 [  6   3]]
0.6613756613756614
Neural Net
[[189   0]
 [  9   0]]
0.5
Naive Bayes
[[ 36 153]
 [  0   9]]
0.5952380952380952
AdaBoost
[[188   1]
 [  7   2]]
0.6084656084656085
Gaussian Process
[[189   0]
 [  9   0]]
0.5
(599, 2689)
Training 10 different classifiers on data after applying CAR filter


C:\Users\Nour Al-Hoda\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic Regression
[[106  83]
 [  0   9]]
0.7804232804232805
Nearest Neighbors
[[189   0]
 [  9   0]]
0.5
Linear SVM
[[187   2]
 [  3   6]]
0.828042328042328
Gradient Boosting Classifier
[[186   3]
 [  6   3]]
0.6587301587301587
Decision Tree
[[182   7]
 [  6   3]]
0.6481481481481481
Random Forest
[[187   2]
 [  7   2]]
0.6058201058201058
Neural Net
[[175  14]
 [  4   5]]
0.7407407407407408
Naive Bayes
[[ 36 153]
 [  0   9]]
0.5952380952380952
AdaBoost
[[188   1]
 [  7   2]]
0.6084656084656085
Gaussian Process
[[189   0]
 [  9   0]]
0.5
Training 10 different classifiers, and different PCA components without CAR filter
Results for  15  PCA components


...

Neural Net
[[182   0]
 [  4   2]]
0.6666666666666666
Naive Bayes
[[182   0]
 [  6   0]]
0.5
AdaBoost
[[181   1]
 [  4   2]]
0.6639194139194139
Gaussian Process
[[182   0]
 [  6   0]]
0.5
Results for  375  PCA components
Logistic Regression
[[182   0]
 [  0   6]]
1.0
Nearest Neighbors
[[182   0]
 [  5   1]]
0.5833333333333334
Linear SVM
[[182   0]
 [  2   4]]


C:\Users\Nour Al-Hoda\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8333333333333333
Gradient Boosting Classifier
[[178   4]
 [  4   2]]
0.6556776556776556
Decision Tree
[[175   7]
 [  4   2]]
0.6474358974358974
Random Forest
[[177   5]
 [  6   0]]
0.48626373626373626


C:\Users\Nour Al-Hoda\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Neural Net
[[182   0]
 [  2   4]]
0.8333333333333333
Naive Bayes
[[182   0]
 [  6   0]]
0.5
AdaBoost
[[181   1]
 [  4   2]]
0.6639194139194139
Gaussian Process
[[182   0]
 [  6   0]]
0.5
